# 2025 데이터 크리에이터 캠프

@PHASE: Mission 1

@TEAM: 최후의 인공지능

## Check GPU Availability

In [ ]:
!nvidia-smi

In [ ]:
# Set CUDA Device Number
DEVICE_NUM = 0
ADDITIONAL_GPU = 0

from os import environ
environ["CUDA_VISIBLE_DEVICES"] = ",".join([f"{i+DEVICE_NUM}" for i in range(0, ADDITIONAL_GPU+1)])
environ["CUDA_VISIBLE_DEVICES"]

## Imports

In [ ]:
from os import path

from creator_camp.datasets import KompsatDatasetForObjectDetection, DatasetHolder

import torch
from torch.utils.data import Dataset
from torchvision import transforms
from transformers import Trainer, TrainingArguments
from accelerate import Accelerator, notebook_launcher

from supervision.metrics.mean_average_precision import MeanAveragePrecision
from supervision.detection.core import Detections

import wandb
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

In [ ]:
if torch.cuda.is_available():
    if ADDITIONAL_GPU:
        device = torch.device("cuda")
    else:
        device = torch.device(f"cuda")  # torch.device(f"cuda:{DEVICE_NUM}")
else:
    device = torch.device("cpu")
    DEVICE_NUM = -1

print(f"INFO: Using device - {device}" + (f":{DEVICE_NUM}" if ADDITIONAL_GPU else ""))

In [ ]:
PROJECT_NAME = "Mission_1"
RUN_NAME = "RT-DETR"

# WandB Initialization
wandb.init(project=PROJECT_NAME, name=RUN_NAME)

## Define Dataset

In [ ]:
DATA_ROOT = path.join(".", "data")

kompstats = DatasetHolder(train=KompsatDatasetForObjectDetection(root=DATA_ROOT, train=True), valid=KompsatDatasetForObjectDetection(root=DATA_ROOT, train=False))

In [ ]:
kompstats.train[0]

In [ ]:
rgb_image, _ = kompstats.train[0]
fig, axes = plt.subplots(1, 1, figsize=(6, 6))

axes.imshow(rgb_image)
axes.set_title('Image')
axes.axis('off')

plt.tight_layout()
plt.show()

## DataLoader

In [ ]:
# Set Batch Size
BATCH_SIZE = 4, 8, 8, 8  # Local
#BATCH_SIZE = 32, 64, 64, 32  # A100

# Dataset Configs
NUM_CLASSES = 1

print(f"INFO: Set batch size - Train: {BATCH_SIZE[0]}, Valid: {BATCH_SIZE[1]}, Test: {BATCH_SIZE[2]}")

In [ ]:
def collate_fn(batch, preprocessor):
    images = [item[0] for item in batch]
    targets = []
    evals = []

    for annotations in [item[1] for item in batch]:
        attrs = annotations['file_attributes']
        targets.append(dict(image_id=len(targets), annotations=[dict(
            bbox=reg['xywh'],  # COCO format
            category_id=0,
            area=attrs['img_width'] * attrs['img_height'],
            iscrowd=0
        ) for reg in annotations['regions']]))
        evals.append([reg['xyxy'] for reg in annotations['regions']])

    processed = preprocessor(images=images, annotations=targets, return_tensors="pt")  # bbox converted to cxcywh format & normalized
    for label, bbox in zip(processed['labels'], evals):
        label['eval_bbox'] = torch.tensor(bbox) if len(bbox) else torch.empty((0, 4), dtype=torch.float32)  # add xyxy bbox for evaluation
    return processed

## Load Model

In [ ]:
from transformers import RTDetrForObjectDetection, RTDetrImageProcessorFast, RTDetrConfig
from transformers.image_utils import AnnotationFormat

In [ ]:
reference_model_id = "PekingU/rtdetr_r101vd"

# Load the reference model configuration
reference_config = RTDetrConfig.from_pretrained(reference_model_id, torch_dtype=torch.float32, return_dict=True)
reference_config.num_labels = NUM_CLASSES + 1

# Set the image size and preprocessor size
reference_config.image_size = 640

# Load the reference model image processor
reference_preprocessor = RTDetrImageProcessorFast.from_pretrained(reference_model_id)
reference_preprocessor.format = AnnotationFormat.COCO_DETECTION  # COCO Format / Detection BBOX Format
reference_preprocessor.size = {"height": 640, "width": 640}
reference_preprocessor.do_resize = True
# ---
test_img_size = 1024
test_preprocessor = RTDetrImageProcessorFast.from_pretrained(reference_model_id)
test_preprocessor.format = AnnotationFormat.COCO_DETECTION  # COCO Format / Detection BBOX Format
test_preprocessor.size = {"height": test_img_size, "width": test_img_size}
test_preprocessor.do_resize = True

In [ ]:
model = RTDetrForObjectDetection.from_pretrained(reference_model_id, config=reference_config, torch_dtype=torch.float32, ignore_mismatched_sizes=True)
model.to(device)

In [ ]:
from transformers.trainer_utils import EvalPrediction
from dataclasses import dataclass


@dataclass
class ModelOutput:
    logits: torch.Tensor
    pred_boxes: torch.Tensor


def map_compute_metrics(preprocessor=reference_preprocessor, threshold=0.0):
    map_metric = MeanAveragePrecision()
    post_process = preprocessor.post_process_object_detection

    def calc(eval_pred: EvalPrediction, compute_result=False):
        nonlocal map_metric

        if compute_result:
            m_ap = map_metric.compute()
            map_metric.reset()

            return {
                "mAP@0.50:0.95": m_ap.map50_95,
                "mAP@0.50": m_ap.map50,
                "mAP@0.75": m_ap.map75
            }
        else:
            preds = ModelOutput(*eval_pred.predictions[1:3])
            labels = eval_pred.label_ids
            sizes = [label['orig_size'].cpu().tolist() for label in labels]

            results = post_process(preds, target_sizes=sizes, threshold=threshold)
            predictions = [Detections.from_transformers(result) for result in results]
            targets = [Detections(
                xyxy=label['eval_bbox'].detach().cpu().numpy(),
                class_id=label['class_labels'].detach().cpu().numpy(),
            ) for label in labels]

            map_metric.update(predictions=predictions, targets=targets)
            return {}
    return calc, map_metric

## Train

In [ ]:
# Set Epoch Count & Learning Rate
EPOCHS = 5
REAL_BATCH = BATCH_SIZE[-1]
LEARNING_RATE = 1e-4

training_args = TrainingArguments(
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    weight_decay=0.1,
    max_grad_norm=0.5,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE[0],
    per_device_eval_batch_size=BATCH_SIZE[1],
    gradient_accumulation_steps=REAL_BATCH//BATCH_SIZE[0],
    eval_accumulation_steps=BATCH_SIZE[1],
    batch_eval_metrics=True,
    remove_unused_columns=False,
    optim="adamw_torch",
    eval_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    eval_steps=100,
    save_steps=100,
    logging_steps=100,
    save_total_limit=100,
    load_best_model_at_end=True,
    metric_for_best_model="mAP@0.50",
    greater_is_better=True,
    #metric_for_best_model="eval_loss",
    #greater_is_better=False,
    report_to="wandb",
    output_dir="./results/"+RUN_NAME,
    logging_dir="./logs/"+RUN_NAME,
    run_name=RUN_NAME,
    bf16=True,
)

testing_args = TrainingArguments(
    per_device_eval_batch_size=BATCH_SIZE[2],
    batch_eval_metrics=True,
    remove_unused_columns=False,
)

In [ ]:
from functools import partial

compute_metrics, compute_results = map_compute_metrics(preprocessor=reference_preprocessor)
test_compute_metrics, test_compute_results = map_compute_metrics(preprocessor=test_preprocessor)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=kompstats.train,
    eval_dataset=kompstats.valid,
    data_collator=partial(collate_fn, preprocessor=reference_preprocessor),
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=30)]
)

tester = Trainer(
    model=model,
    args=testing_args,
    eval_dataset=kompstats.valid,
    data_collator=partial(collate_fn, preprocessor=test_preprocessor),
    compute_metrics=test_compute_metrics
)

In [ ]:
def start_train():
    accelerator = Accelerator()
    while True:
        try:
            try:
                print("INFO: Trying to resume from previous checkpoint")
                compute_results.reset()
                trainer.train(resume_from_checkpoint=True)
            except Exception as e:
                if "No valid checkpoint found" in str(e):
                    print(f"ERROR: Failed to resume from checkpoint - {e}")
                    print("INFO: Starting training from scratch")
                    compute_results.reset()
                    trainer.train(resume_from_checkpoint=False)
        except Exception as e:
            if "CUDA" in str(e):
                print(f"ERROR: CUDA Error - {e}")
                trainer.train()
            else:
                raise e

In [ ]:
if ADDITIONAL_GPU:
    notebook_launcher(start_train, args=(), num_processes=ADDITIONAL_GPU)
else:
    start_train()

## Evaluate

In [ ]:
trainer.evaluate()

In [ ]:
tester.evaluate()

In [ ]:
checkpoint = 31100

In [ ]:
try:
    model = RTDetrForObjectDetection.from_pretrained(f"{training_args.output_dir}/checkpoint-{checkpoint}/", torch_dtype=torch.float32, return_dict=True, local_files_only=True)
    model.to(device)
except Exception:
    pass